# <img style="float: left; padding-right: 10px; width: 45px" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png"> CS207 Systems Development for Computational Science: 
## autodiff Demo



**Harvard University**<br/>
**Fall 2018**<br/>
**Team Members**: Will Claybaugh, Erin Williams, and Bruce Xiong

<hr style="height:2pt">

In [1]:
import sys
#NOTE: You must replace 'C:/Users/erina/cs207-FinalProject/' below with the local path to autodiff on your machine
sys.path.insert(0, "C:/Users/erina/cs207-FinalProject/")
import autodiff as ad

def f(a,b):
    return a/b*ad.sin(a*b)

out = f(ad.DualNumber('x',2),ad.DualNumber('y',3))

ModuleNotFoundError: No module named 'autodiff'

In [2]:
out.value
# Should return -0.416146837

NameError: name 'out' is not defined

In [3]:
out.derivatives['x']
# Should return -1.09115691

1.82720207390109

In [4]:
out.derivatives['y']
# Should return -8.63832555

1.3423193818002492